In [1]:
from QASMBench.interface.qiskit import QASMBenchmark

# path to the root directory of QASMBench
path = "QASMBench"
# selected category for QASMBench
category = "large" 
# select only the circuits with the number of qubits in the list
num_qubits_list = list(range(100, 101))
# whether to remove the final measurement in the circuit
remove_final_measurements = True
# whether to use qiskit.transpile() to transpile the circuits (note: must provide qiskit backend)
do_transpile = True  # Changed to True for transpiling circuits
# arguments for qiskit.transpile(). backend should be provided at least
transpile_args = { 'basis_gates': ['u', 'cp']}
bm = QASMBenchmark(path, category, num_qubits_list=num_qubits_list, do_transpile=do_transpile, **transpile_args, remove_final_measurements=remove_final_measurements)

# circ_list = bm.circ_list
circ_name_list = bm.circ_name_list
for name in circ_name_list:
    print(name)

QV_n100


In [2]:
import os
import json
import numpy as np
from qiskit import transpile
from disqco.graphs.GCP_hypergraph import QuantumCircuitHyperGraph
from disqco.parti.FM.FM_main import *
from disqco.parti.FM.multilevel_FM import *
from disqco.graphs.quantum_network import grid_coupling, linear_coupling, random_coupling

###############################################################################
# Set up JSON file for storing *all* iteration results (detailed data)
###############################################################################
detailed_filename = "benchmark_results_MLFM-R_QASM_grid.json"

if os.path.exists(detailed_filename):
    with open(detailed_filename, "r") as f:
        detailed_results = json.load(f)
else:
    detailed_results = []

###############################################################################
# Set up JSON file for *aggregated* results (mean cost/time)
###############################################################################

means_filename = "benchmark_means_MLFM-R_QASM_grid.json"

if os.path.exists(means_filename):
    with open(means_filename, "r") as f:
        mean_results = json.load(f)
else:
    mean_results = []

num_partition_list = [6,8]

circ_name_list = ['QV_n100']

for num_partitions in num_partition_list:

    for i, circ_name in enumerate(circ_name_list):
        i = i
        circ = bm.get(circ_name)
        num_qubits = circ.num_qubits
        circuit = circ
        circuit = transpile(circuit, basis_gates = ['u', 'cp'])
        if circuit.depth() < 1000:
            print(f"Starting benchmark for {num_qubits} qubits, {num_partitions} partitions, circuit: {circ_name_list[i]}")

            base_graph = QuantumCircuitHyperGraph(circuit, group_gates=True)
            depth = base_graph.depth
            qpu_info = [int(num_qubits / num_partitions) + 1 for _ in range(num_partitions)]
            coupling = grid_coupling(num_partitions)
            network = QuantumNetwork(qpu_info, coupling)
            initial_assignment = set_initial_partitions(network, num_qubits, depth+2)
            num_qubits = circ.num_qubits

            
            iteration_data = []
            for iteration in range(1):

                assignment_list_r, cost_list_r, time_list_r = MLFM_recursive_hetero(base_graph,
                                                                                    initial_assignment,  
                                                                                    qpu_info, 
                                                                                    limit = num_qubits, 
                                                                                    costs = {},
                                                                                    network = network,)
                
                total_time_r = sum(time_list_r)
                min_cost_r = min(cost_list_r)


                result_entry = {
                    "circuit_name": circ_name_list[i],
                    "num_qubits": num_qubits,
                    "num_partitions": num_partitions,
                    "iteration": iteration,
                    "r_cost":  min_cost_r,
                    "time_r": total_time_r,
                }
                
                detailed_results.append(result_entry)
                iteration_data.append(result_entry)
                
                # Update detailed JSON right away
                with open(detailed_filename, "w") as f:
                    json.dump(detailed_results, f, indent=2)
            
            # ---------------------------------------------------------------------
            # After 5 iterations, compute the means and log them
            # ---------------------------------------------------------------------

            r_cost_list = [x["r_cost"] for x in iteration_data]
            r_time_list = [x["time_r"] for x in iteration_data]
            

            mean_r_cost = float(np.mean(r_cost_list))
            mean_r_time = float(np.mean(r_time_list))
            
            print("=============================================")
            print(f"Finished 10 iterations for:")
            print(f"  # Qubits: {num_qubits}, # Partitions: {num_partitions}, circuit: {circ_name_list[i]}")
            print("Mean Costs:")

            print(f"  Recursive:{mean_r_cost:.3f}")
            print("Mean Times (s):")

            print(f"  Recursive:{mean_r_time:.3f}")
            print("=============================================")
            
            # Store the aggregated means in a separate JSON
            mean_entry = {
                "circuit_name": circ_name_list[i],
                "num_qubits": num_qubits,
                "num_partitions": num_partitions,
                "mean_r_cost": mean_r_cost,
                "mean_r_time": mean_r_time,
            }
            
            mean_results.append(mean_entry)
            
            # Update the means JSON file
            with open(means_filename, "w") as f:
                json.dump(mean_results, f, indent=2)

print("Benchmarking completed. Detailed results saved to", detailed_filename)
print("Aggregated means saved to", means_filename)

Starting benchmark for 100 qubits, 6 partitions, circuit: QV_n100
Finished 10 iterations for:
  # Qubits: 100, # Partitions: 6, circuit: QV_n100
Mean Costs:
  Recursive:7203.000
Mean Times (s):
  Recursive:122.741
Starting benchmark for 100 qubits, 8 partitions, circuit: QV_n100
Finished 10 iterations for:
  # Qubits: 100, # Partitions: 8, circuit: QV_n100
Mean Costs:
  Recursive:9470.000
Mean Times (s):
  Recursive:170.050
Benchmarking completed. Detailed results saved to benchmark_results_MLFM-R_QASM_grid.json
Aggregated means saved to benchmark_means_MLFM-R_QASM_grid.json


In [3]:
import os
import json
import numpy as np
from qiskit import transpile
from disqco.graphs.GCP_hypergraph import QuantumCircuitHyperGraph
from disqco.parti.FM.FM_main import *
from disqco.parti.FM.multilevel_FM import *
from disqco.graphs.quantum_network import grid_coupling, linear_coupling, random_coupling

###############################################################################
# Set up JSON file for storing *all* iteration results (detailed data)
###############################################################################
detailed_filename = "benchmark_results_MLFM-R_QASM_linear.json"

if os.path.exists(detailed_filename):
    with open(detailed_filename, "r") as f:
        detailed_results = json.load(f)
else:
    detailed_results = []

###############################################################################
# Set up JSON file for *aggregated* results (mean cost/time)
###############################################################################

means_filename = "benchmark_means_MLFM-R_QASM_linear.json"

if os.path.exists(means_filename):
    with open(means_filename, "r") as f:
        mean_results = json.load(f)
else:
    mean_results = []

num_partition_list = [6,8]

circ_name_list = ['QV_n100']

for num_partitions in num_partition_list:

    for i, circ_name in enumerate(circ_name_list):
        i = i
        circ = bm.get(circ_name)
        num_qubits = circ.num_qubits
        circuit = circ
        circuit = transpile(circuit, basis_gates = ['u', 'cp'])
        if circuit.depth() < 1000:
            print(f"Starting benchmark for {num_qubits} qubits, {num_partitions} partitions, circuit: {circ_name_list[i]}")

            base_graph = QuantumCircuitHyperGraph(circuit, group_gates=True)
            depth = base_graph.depth
            qpu_info = [int(num_qubits / num_partitions) + 1 for _ in range(num_partitions)]
            coupling = linear_coupling(num_partitions)
            network = QuantumNetwork(qpu_info, coupling)
            initial_assignment = set_initial_partitions(network, num_qubits, depth+2)
            num_qubits = circ.num_qubits

            
            iteration_data = []
            for iteration in range(1):

                assignment_list_r, cost_list_r, time_list_r = MLFM_recursive_hetero(base_graph,
                                                                                    initial_assignment,  
                                                                                    qpu_info, 
                                                                                    limit = num_qubits, 
                                                                                    costs = {},
                                                                                    network = network,)
                
                total_time_r = sum(time_list_r)
                min_cost_r = min(cost_list_r)


                result_entry = {
                    "circuit_name": circ_name_list[i],
                    "num_qubits": num_qubits,
                    "num_partitions": num_partitions,
                    "iteration": iteration,
                    "r_cost":  min_cost_r,
                    "time_r": total_time_r,
                }
                
                detailed_results.append(result_entry)
                iteration_data.append(result_entry)
                
                # Update detailed JSON right away
                with open(detailed_filename, "w") as f:
                    json.dump(detailed_results, f, indent=2)
            
            # ---------------------------------------------------------------------
            # After 5 iterations, compute the means and log them
            # ---------------------------------------------------------------------

            r_cost_list = [x["r_cost"] for x in iteration_data]
            r_time_list = [x["time_r"] for x in iteration_data]
            

            mean_r_cost = float(np.mean(r_cost_list))
            mean_r_time = float(np.mean(r_time_list))
            
            print("=============================================")
            print(f"Finished 10 iterations for:")
            print(f"  # Qubits: {num_qubits}, # Partitions: {num_partitions}, circuit: {circ_name_list[i]}")
            print("Mean Costs:")

            print(f"  Recursive:{mean_r_cost:.3f}")
            print("Mean Times (s):")

            print(f"  Recursive:{mean_r_time:.3f}")
            print("=============================================")
            
            # Store the aggregated means in a separate JSON
            mean_entry = {
                "circuit_name": circ_name_list[i],
                "num_qubits": num_qubits,
                "num_partitions": num_partitions,
                "mean_r_cost": mean_r_cost,
                "mean_r_time": mean_r_time,
            }
            
            mean_results.append(mean_entry)
            
            # Update the means JSON file
            with open(means_filename, "w") as f:
                json.dump(mean_results, f, indent=2)

print("Benchmarking completed. Detailed results saved to", detailed_filename)
print("Aggregated means saved to", means_filename)

Starting benchmark for 100 qubits, 6 partitions, circuit: QV_n100
Finished 10 iterations for:
  # Qubits: 100, # Partitions: 6, circuit: QV_n100
Mean Costs:
  Recursive:11088.000
Mean Times (s):
  Recursive:123.893
Starting benchmark for 100 qubits, 8 partitions, circuit: QV_n100
Finished 10 iterations for:
  # Qubits: 100, # Partitions: 8, circuit: QV_n100
Mean Costs:
  Recursive:16010.000
Mean Times (s):
  Recursive:174.598
Benchmarking completed. Detailed results saved to benchmark_results_MLFM-R_QASM_linear.json
Aggregated means saved to benchmark_means_MLFM-R_QASM_linear.json


In [ ]:
circ = bm.get('cc_n32')

In [ ]:
from MLFM_GCP.utils.qiskit_to_op_list import *

op_list = circuit_to_gate_layers(circ)

for layer in op_list:
    for op in layer:
        print(op)

print(circ)

ModuleNotFoundError: No module named 'MLFM_GCP'